### Begin hier
Begin met het runnen van deze blokken code.  
Deze zijn verantwoordelijk voor imports, setup, dependencies en andere zaken. 

In [ ]:
# Imports
import install_deps as deps
choiche = input("Install Deps? (choose y to continue) ")
if choiche == "y":
    deps.unattended_install()
else:
    pass

import pandas as pd
import cv2
import numpy as np
from matplotlib import pyplot as plt
import data_generator as dgen

print("Succes loading Imports")

In [ ]:
# Functions
def get_file_name(path):
    file_name = path[path.rfind('/') + 1:]
    # Splits out the file-type designation
    file_name, file_extension = file_name.split('.')
    return file_name
    
print("Succes loading Functions")

### Heb je al data?
Als er al trainingsdata bestaat, schrijf dan simpelweg de filename bij het betreffende blok.  
Run anders het blok waarin de data nog word gegenereerd. 

In [ ]:
"""Ik heb geen data"""
# Start here if no data has been generated.

print("Provide the path to a video file.\n"
      "  (Example: C:/Users/admin/video/video.mp4)")

vid_path = input("Provide a video path: ")

try:
      vid_name = get_file_name(vid_path)
      print(f"Provided path: '{vid_path}',\nfound filename: '{vid_name}'")
      print("")
except Exception as e:
      print(f"Something went wrong, did you not provide a path?\n{e}")

try:
      generator = dgen.Generator(vid_path)
      generator.generate_data()
      print(f"Video has been processed and exported to {generator.file_name}__trainingsData.mp4")
      print(f"Data generated and exported to {generator.file_name}_trainingsData.json")
      print("")
except Exception as e:
      print(f"Something went wrong, any generated data can be dismissed.\n{e}")

try: 
      data_path = f"generateData_OUTPUT/{vid_name}_trainingsData.json"
      frame_data = pd.read_json(data_path, orient='index')
except Exception as e:
      print(f"Something went wrong.\n {e}")


print("The data-table was loaded. \nTotal Table size: ")
print(f"Rows: {frame_data.shape[0]}, Columns: {frame_data.shape[1]}")

In [ ]:
"""Ik heb al trainingsdata. """
# Start here if data has already been generated.
print("Provide the filename to a file in "
      "the generateData_OUTPUT folder.\n"
      "Example: perfect_example_trainingsData")

try:
      data_path = input("Filename: ")
      vid_name = get_file_name(data_path)
      data_path = f"generateData_OUTPUT/{vid_name}.json"
      print(f"Provided path: {data_path}")
except Exception as e:
      print(f"Something went wrong, did you not provide a path?\n{e}")

try: 
      frame_data = pd.read_json(data_path, orient='index')
except Exception as e:
      print(f"Something went wrong.\n {e}")

print("\nTotal Table size: ")
print(f"Rows: {frame_data.shape[0]}, Columns: {frame_data.shape[1]}")

In [ ]:
# cycle_data = pd.read_csv("dataset bike/220316_1023_JOS.csv", sep=";")
# cycle_data.head()
#
# maxhr = max(cycle_data["idHeartrate"])
# cycle_data['heartRateZone'] = cycle_data.apply(
#     lambda row: 5 if (row.idHeartrate > (maxhr * 0.9)) else (
#         4 if (row.idHeartrate > (maxhr * 0.8)) else (
#             3 if (row.idHeartrate > (maxhr * 0.7)) else (
#                 2 if (row.idHeartrate > (maxhr * 0.6)) else (
#                     1 if (row.idHeartrate > (maxhr * 0.5)) else (0))))), axis=1)

In [ ]:
frame_data.head()

In [ ]:
X = frame_data.drop(columns=['TS', 'HR', 'PWR', 'heartRateZone'])
Y = frame_data["heartRateZone"]

#print(y)

In [69]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
from keras.layers import Layer
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dense, Input, Concatenate, Lambda

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
Y_train.head()

In [70]:
inputTensor = Input(shape=(1434,))
layers = []

for i in range(0, 1431, 3):
    layers.append(Lambda(lambda x: x[:,i: i+3], output_shape=((3,)))(inputTensor))
for i in range(0, 477):
    layers[i] = Dense(1)(layers[i])
coordslayer = Concatenate()(layers)

hidden1 = Dense(477*2, activation='relu')(coordslayer)
hidden2 = Dense(477, activation='relu')(hidden1)
hidden3 = Dense(240, activation='relu')(hidden2)
hidden4 = Dense(100, activation='relu')(hidden3)
hidden5 = Dense(20, activation='relu')(hidden4)

outputTensor = Dense(5, activation='sigmoid')(hidden5)
model = Model(inputs=inputTensor, outputs=outputTensor)

In [ ]:
print(model.summary())

In [73]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, Y, epochs=150, batch_size=10)

In [ ]:
# evaluate the keras model
_, accuracy = model.evaluate(X, Y)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
x = np.array([[0,00,000], [1,11,111], [2,22,222], [3,33,333]])
print(x[:,:])
print(x[:,0:3])
g = lambda g: g[:,2:4]
print(g(x))